In [30]:
from os import listdir
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import DataLoader

import Sources.DataNasaFan as DataTool
from Sources.ModelsNasaFan import RNNModel, GRUModel, LSTMModel

import AdversarialRulLib.Utils as UtilsTool
import AdversarialRulLib.Attacks as AttacksTool
import AdversarialRulLib.Defense as DefenseTool

In [31]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

TrainMode = 0

DatasetPath = 'DatasetNasaFan/'
ModelPath = 'Models/'
Window = 80

In [32]:
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {Device}")

Using device: cuda


In [33]:
listdir(DatasetPath)

DfTrain = pd.read_csv(DatasetPath + 'train_FD001.txt', header = None, sep = ' ')
DfTest = pd.read_csv(DatasetPath + 'test_FD001.txt', header = None, sep = ' ')
RulTest = pd.read_csv(DatasetPath + 'RUL_FD001.txt', header = None)

In [34]:
DfTrain, DfTest, RulTest = DataTool.SetDatasets(DfTrain, DfTest, RulTest)

In [35]:
MinMaxDict = DataTool.GetMinMaxDictionnary(DfTrain)
NdfTrain  = DataTool.NormalizeDataset(DfTrain, MinMaxDict)    
NdfTest  = DataTool.NormalizeDataset(DfTest, MinMaxDict)   

In [36]:
NbFeatures = len([c for c in NdfTrain.columns if 's' in c])

In [37]:
np.random.seed(5)
torch.manual_seed(5)

Units = np.arange(1,101)
TrainUnits = list(np.random.choice(Units, 80, replace = False))
ValUnits = list(set(Units) - set(TrainUnits))

TrainData = NdfTrain[NdfTrain['unit'].isin(TrainUnits)].copy()
ValData = NdfTrain[NdfTrain['unit'].isin(ValUnits)].copy()

TrainIndices = list(TrainData[(TrainData['rul'] >= (Window - 1)) & (TrainData['time'] > 10)].index)
ValIndices = list(ValData[(ValData['rul'] >= (Window - 1)) & (ValData['time'] > 10)].index)

TrainDataset = DataTool.data(TrainIndices, NdfTrain, Window)
ValidationDataset = DataTool.data(ValIndices, NdfTrain, Window)

UnitBySize = NdfTest.groupby('unit')['time'].max().tolist()
Units = []
for i in range(len(UnitBySize)):
    if UnitBySize[i] > Window:
        Units.append(i+1)

TestDataset = DataTool.test(NdfTest, Window)

In [38]:
print("Train", len(TrainDataset))
print("Val", len(ValidationDataset))
print("Test", len(TestDataset))

Train 9615
Val 2116
Test 77


In [39]:
TrainLoader = DataLoader(TrainDataset, batch_size = 64, shuffle = True)
ValidationLoader = DataLoader(ValidationDataset, batch_size = len(ValIndices), shuffle = True)
TestLoader = DataLoader(TestDataset, batch_size = 300)

In [40]:
if TrainMode == 1:
    epochs = 40
    learning_rate= 0.01
    
    MyRNNModel = RNNModel().to(Device)
    UtilsTool.InitModel(MyRNNModel, Device)
    UtilsTool.TrainModel(MyRNNModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyRNNModel, ModelPath + 'MyRNNmodel.pth')

    MyGRUModel = GRUModel().to(Device)
    UtilsTool.InitModel(MyGRUModel, Device)
    UtilsTool.TrainModel(MyGRUModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyGRUModel, ModelPath + 'MyGRUmodel.pth')

    MyLSTMModel = LSTMModel().to(Device)
    UtilsTool.InitModel(MyLSTMModel, Device)
    UtilsTool.TrainModel(MyLSTMModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)     
    torch.save(MyLSTMModel, ModelPath + 'MyLSTMmodel.pth')
    
else:     
    MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth').to(Device)
    MyGRUModel = torch.load(ModelPath + 'MyGRUmodel.pth').to(Device)
    MyLSTMModel = torch.load(ModelPath + 'MyLSTMmodel.pth').to(Device)


C:\Users\pfmai\AppData\Local\Temp\ipykernel_9228\3195802369.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth').to(Dev

![](./animation3.gif)

In [41]:
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyRNNModel, TestLoader, Device)
print(f'RNN: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'GRU: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyLSTMModel, TestLoader, Device)
print(f'LSTM: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')

RNN: MSE:447.76, L1:15.27, RMSE:21.16
GRU: MSE:206.27, L1:9.65, RMSE:14.36
LSTM: MSE:360.85, L1:13.55, RMSE:19.0


In [42]:
TestLoaderAttack = DataLoader(TestLoader.dataset, batch_size=300, shuffle=False)
X, y = next(iter(TestLoaderAttack))
X, y = X.to(Device), y.to(Device)

In [43]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002
    },
    
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}

print("GRU")
GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2,  GRUAdversarialDataL0 = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()

GRU
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 41.70382308959961, 'RMSE_adversarial': 0.011127319186925888, 'RMSE_pred': 29.44731903076172, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 40.320762634277344, 'RMSE_adversarial': 0.010623768903315067, 'RMSE_pred': 31.117568969726562, 'L0': tensor(24.)}
L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 22.969768524169922, 'RMSE_adversarial': 0.011934179812669754, 'RMSE_pred': 53.648399353027344, 'L0': tensor(24.)}
L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.945420265197754, 'RMSE_adversarial': 0.06632962077856064, 'RMSE_pred': 72.83966827392578, 'L0': tensor(2.0519)}



In [44]:
AttackParameters = {
        "Epsilon": 0.02
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingFgsm = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Fgsm", AttackParameters = AttackParameters,  Epochs = 80, Device="cuda", Verbose=0)

AttackParameters = {
        "Iterations": 30,
        "Epsilon": 0.001
    }
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingBim = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Bim", AttackParameters = AttackParameters,  Epochs = 80, Device="cuda", Verbose=0)

AttackParameters = {
        "Iterations": 50,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingL2 = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "L2", AttackParameters = AttackParameters,  Epochs = 80 , Device="cuda", Verbose=0)

100%|██████████| 80/80 [3:03:31<00:00, 137.64s/it]  


In [45]:
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_EnsembleInputSqueezing = DefenseTool.EnsembleInputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0, MaxCoefficient=2)
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputSqueezing = DefenseTool.InputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputRandomization = DefenseTool.InputRandomization(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputPurificationAutoEncoder = DefenseTool.InputPurificationAutoEncoder(CurrentModel, TrainLoader, ValidationLoader, TestLoader, EpochsAutoEncoder = 30, EpochsModel = 30, Device="cuda", Verbose=0)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [16:27<00:00, 32.92s/it]


In [46]:
AttackParameters = {
        "Epsilon": 0.01
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
DetectionModelFgsm = DefenseTool.GenerateDetectionModel(CurrentModel, TrainLoader, ValidationLoader, Device="cuda", Epochs = 30, LearningRate = 0.001,AttackParameters=AttackParameters, AdversarialMethod="Fgsm", Verbose=0)

AttackParameters = {
        "Iterations": 30,
        "Epsilon": 0.001
}

DetectionModelBim = DefenseTool.GenerateDetectionModel(CurrentModel, TrainLoader, ValidationLoader, Device="cuda", Epochs = 30, LearningRate = 0.001,AttackParameters=AttackParameters, AdversarialMethod="Bim", Verbose=0)

AttackParameters = {
        "Iterations": 50,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }

DetectionModelL2 = DefenseTool.GenerateDetectionModel(CurrentModel, TrainLoader, ValidationLoader, Device="cuda", Epochs = 30, LearningRate = 0.001,AttackParameters=AttackParameters, AdversarialMethod="L2", Verbose=0)

100%|██████████| 30/30 [1:45:36<00:00, 211.20s/it]


In [47]:
Model_Distillation = DefenseTool.DefensiveDistillation(MyGRUModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

100%|██████████| 30/30 [13:29<00:00, 26.99s/it]


In [48]:
RFModel, XGBModel = DefenseTool.EnsembleMethods(TrainLoader, ValidationLoader, TestLoader)

In [49]:
mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'Classicial : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingFgsm, TestLoader, Device)
print(f'Adversarial training FGSM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingBim, TestLoader, Device)
print(f'Adversarial training BIM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingL2, TestLoader, Device)
print(f'Adversarial training L2: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_EnsembleInputSqueezing, TestLoader, Device)
print(f'Ensemble squeezing : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputSqueezing, TestLoader, Device)
print(f'Squeezing: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputRandomization, TestLoader, Device)
print(f'Randomization: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputPurificationAutoEncoder, TestLoader, Device)
print(f'AutoEncoder: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_Distillation, TestLoader, Device)
print(f'Distillation: RMSE:{round(Rrmse,2)}')

DetectionRate = torch.round(DetectionModelFgsm(X.float())).sum()/len(X.float())
print("Detection rate: RMSE ", DetectionRate.item())

DetectionRate = torch.round(DetectionModelBim(X.float())).sum()/len(X.float())
print("Detection rate: RMSE ", DetectionRate.item())

DetectionRate = torch.round(DetectionModelL2(X.float())).sum()/len(X.float())
print("Detection rate: RMSE ", DetectionRate.item())

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.Flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(RFModel.predict(XTest), y_test))
print(f'Random Forest test RMSE: {MyRMSE:.2f}')

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.Flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(XGBModel.predict(XTest), y_test))
print(f'XGB RMSE: {MyRMSE:.2f}')

Classicial : RMSE:14.36
Adversarial training FGSM: RMSE:21.86
Adversarial training BIM: RMSE:22.15
Adversarial training L2: RMSE:23.24
Ensemble squeezing : RMSE:19.76
Squeezing: RMSE:17.56
Randomization: RMSE:21.55
AutoEncoder: RMSE:16.76
Distillation: RMSE:15.98
Detection rate: RMSE  0.0
Detection rate: RMSE  0.0
Detection rate: RMSE  0.6623376607894897
Random Forest test RMSE: 14.47
XGB RMSE: 17.05


In [50]:
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2, GRUAdversarialDataL0]
Models = [MyGRUModel, Model_AdvTrainingFgsm, Model_AdvTrainingBim, Model_AdvTrainingL2, Model_EnsembleInputSqueezing, Model_InputSqueezing, Model_InputRandomization, Model_InputPurificationAutoEncoder, Model_Distillation]
ModelsName = ["Classical", "Model_AdvTrainingFgsm", "Model_AdvTrainingBim", "Model_AdvTrainingL2", "Model_EnsembleInputSqueezing", "Model_InputSqueezing", "Model_InputRandomization", "Model_InputPurificationAutoEncoder", "Model_Distillation"]
EnsemblModels = [XGBModel, RFModel]
EnsemblModelsName = [ "XFG", "RF"]

UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)
UtilsTool.DisplayDetection(DetectionModelFgsm, AdversarialExamples)
UtilsTool.DisplayDetection(DetectionModelBim, AdversarialExamples)
UtilsTool.DisplayDetection(DetectionModelL2, AdversarialExamples)
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

Classical
0 RMSE  14.36225414276123
1 RMSE  29.44731903076172
2 RMSE  31.117568969726562
3 RMSE  53.648399353027344
4 RMSE  72.83966827392578

Model_AdvTrainingFgsm
0 RMSE  21.864301681518555
1 RMSE  18.703458786010742
2 RMSE  18.196327209472656
3 RMSE  22.09284782409668
4 RMSE  32.903507232666016

Model_AdvTrainingBim
0 RMSE  22.15067481994629
1 RMSE  20.13359832763672
2 RMSE  20.133724212646484
3 RMSE  23.114789962768555
4 RMSE  31.040386199951172

Model_AdvTrainingL2
0 RMSE  23.24265480041504
1 RMSE  22.021198272705078
2 RMSE  21.771194458007812
3 RMSE  20.7477970123291
4 RMSE  20.522483825683594

Model_EnsembleInputSqueezing
0 RMSE  19.756303787231445
1 RMSE  28.158884048461914
2 RMSE  28.20350456237793
3 RMSE  37.87398910522461
4 RMSE  49.26209259033203

Model_InputSqueezing
0 RMSE  17.564226150512695
1 RMSE  28.724876403808594
2 RMSE  28.12258529663086
3 RMSE  40.24281311035156
4 RMSE  59.04916000366211

Model_InputRandomization
0 RMSE  19.41832733154297
1 RMSE  20.33296585083007

In [51]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.010
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 105,
        "Epsilon": 0.0001
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
print("Classical Model")
_, _, _, _ = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model FGSM")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingFgsm, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model BIM")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingBim, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model L2")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingL2, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model Ensemble squeezing")
_, _, _, _ = AttacksTool.TestAttacks(Model_EnsembleInputSqueezing, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model squeezing")
_, _, _, _ = AttacksTool.TestAttacks(Model_InputSqueezing, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000015,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}

print("Adversarial Distillation")
_, _, _, _ = AttacksTool.TestAttacks(Model_Distillation, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.011
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 140,
        "Epsilon": 0.0001
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000007,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
print("Adversarial Randomization")
_, _, _, _ = AttacksTool.TestAttacks(Model_InputRandomization, X, y, AttacksParameters, Device = Device)
print()
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
print("Adversarial Autoencoder")
_, _, _, _ = AttacksTool.TestAttacks(Model_InputPurificationAutoEncoder, X, y, AttacksParameters, Device = Device)

Classical Model
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 44.0766487121582, 'RMSE_adversarial': 0.0092728640884161, 'RMSE_pred': 27.123380661010742, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 42.32912826538086, 'RMSE_adversarial': 0.00934695452451706, 'RMSE_pred': 29.091541290283203, 'L0': tensor(24.)}
L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 22.969768524169922, 'RMSE_adversarial': 0.011934179812669754, 'RMSE_pred': 53.648399353027344, 'L0': tensor(24.)}
L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.7247250080108643, 'RMSE_adversarial': 0.06574776023626328, 'RMSE_pred': 73.14331817626953, 'L0': tensor(2.0779)}

Adversarial Model FGSM
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 64.87660217285156, 'RMSE_adversarial': 0.008537886664271355, 'RMSE_pred': 15.745138168334961, 'L0': tensor(21.0909)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 63.81242370605469, '

In [57]:
AttacksParameters = {
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.02,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
print("Classical Model")
_, _, _, _ = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model FGSM")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingFgsm, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model BIM")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingBim, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model L2")
_, _, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingL2, X, y, AttacksParameters, Device = Device)

Classical Model
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 44.0766487121582, 'RMSE_adversarial': 0.0092728640884161, 'RMSE_pred': 27.123380661010742, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 42.32912826538086, 'RMSE_adversarial': 0.00934695452451706, 'RMSE_pred': 29.091541290283203, 'L0': tensor(24.)}
L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 23.20207977294922, 'RMSE_adversarial': 0.012014318257570267, 'RMSE_pred': 53.20323181152344, 'L0': tensor(24.)}
L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 2.5976202487945557, 'RMSE_adversarial': 0.06255000829696655, 'RMSE_pred': 74.36075592041016, 'L0': tensor(3.3636)}

Adversarial Model FGSM
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 64.87660217285156, 'RMSE_adversarial': 0.008537886664271355, 'RMSE_pred': 15.745138168334961, 'L0': tensor(21.0909)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 63.81242370605469, 'RM

In [52]:
SurrogateModelXGB = AttacksTool.CreateSurrogateModel(XGBModel, Epochs = 100, Device = "cuda", Verbose = 0)
SurrogateModelRF = AttacksTool.CreateSurrogateModel(RFModel, Epochs = 30, Device = "cuda", Verbose = 0)

100%|██████████| 30/30 [07:41<00:00, 15.37s/it]


In [53]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
A, B, C, D = AttacksTool.TestAttacks(SurrogateModelXGB, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2, GRUAdversarialDataL0, A, B, C, D]
EnsemblModels = [XGBModel]
EnsemblModelNames = ["XGBModel"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
A, B, C, D = AttacksTool.TestAttacks(SurrogateModelRF, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2, GRUAdversarialDataL0, A, B, C, D]
EnsemblModels = [RFModel]
EnsemblModelNames = ["RF"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelNames, AdversarialExamples, y)

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 96.48160552978516, 'RMSE_adversarial': 0.01144420076161623, 'RMSE_pred': 37.34797668457031, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 96.56033325195312, 'RMSE_adversarial': 0.010999324731528759, 'RMSE_pred': 37.41765594482422, 'L0': tensor(24.)}
L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 66.31300354003906, 'RMSE_adversarial': 0.009540744125843048, 'RMSE_pred': 32.20648956298828, 'L0': tensor(24.)}
L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 42.31998825073242, 'RMSE_adversarial': 0.1732914000749588, 'RMSE_pred': 46.81394958496094, 'L0': tensor(7.4026)}
XFG
0 RMSE  17.05215
1 RMSE  20.380913
2 RMSE  20.593645
3 RMSE  41.136745
4 RMSE  43.810425
5 RMSE  16.649055
6 RMSE  15.767556
7 RMSE  39.050587
8 RMSE  73.39851

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 83.72765350341797, 'RMSE_adversarial': 0.011321657337248325, 'RMSE_pred': 27.

In [54]:
SurrogateSqueezing = AttacksTool.CreateSurrogateModelSqueezing(Model_InputSqueezing, Epochs = 300, LearningRate=0.01, Device ="cuda", Verbose = 0)
SurrogateSqueezing2 = AttacksTool.CreateSurrogateModelSqueezing(Model_EnsembleInputSqueezing, Epochs = 300, LearningRate=0.001, Device = "cuda", Verbose = 0)

100%|██████████| 300/300 [4:52:05<00:00, 58.42s/it]     


In [55]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.0001
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0003,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}

A, B, C, D = AttacksTool.TestAttacks(SurrogateSqueezing, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2, GRUAdversarialDataL0, A, B, C, D]
Models = [Model_EnsembleInputSqueezing]
ModelsName = ["Model_EnsembleInputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)


AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "L2": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00002,
    },
    "L0": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.01,
        "k": 2,
        "p": 8,
        "s": 0.1
    } 
}
A, B, C, D = AttacksTool.TestAttacks(SurrogateSqueezing2, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataL2, GRUAdversarialDataL0, A, B, C, D]
Models = [Model_InputSqueezing]
ModelsName = ["Model_InputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 80.4784927368164, 'RMSE_adversarial': 0.011116898618638515, 'RMSE_pred': 32.265769958496094, 'L0': tensor(23.9870)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 78.2060546875, 'RMSE_adversarial': 0.011251232586801052, 'RMSE_pred': 31.521615982055664, 'L0': tensor(23.9740)}
L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.1504223346710205, 'RMSE_adversarial': 0.08472742140293121, 'RMSE_pred': 72.91426086425781, 'L0': tensor(23.9870)}
L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 7.0261454582214355, 'RMSE_adversarial': 0.1305360198020935, 'RMSE_pred': 70.29898071289062, 'L0': tensor(5.5065)}
Model_EnsembleInputSqueezing
0 RMSE  19.756303787231445
1 RMSE  28.158884048461914
2 RMSE  28.20350456237793
3 RMSE  37.87398910522461
4 RMSE  49.26209259033203
5 RMSE  24.028709411621094
6 RMSE  25.935449600219727
7 RMSE  61.20918273925781
8 RMSE  83.63777923583984

FGSM: {'RealRUL': 64.49350

In [56]:
Adv = AttacksTool.L2AttackDetection(MyGRUModel, DetectionModelFgsm, X, y, LearningRate= 0.00002, c = 1, d = 10000, Device = "cuda", Verbose =0)
mse = torch.mean((Adv - X) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")

mse = torch.mean((MyGRUModel(Adv) - y) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")
UtilsTool.DisplayDetection(DetectionModelFgsm, [Adv])

Adv = AttacksTool.L2AttackDetection(MyGRUModel, DetectionModelBim, X, y, LearningRate= 0.00002, c = 1, d = 10000, Device = "cuda", Verbose =0)
mse = torch.mean((Adv - X) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")

mse = torch.mean((MyGRUModel(Adv) - y) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")
UtilsTool.DisplayDetection(DetectionModelBim, [Adv])

Adv = AttacksTool.L2AttackDetection(MyGRUModel, DetectionModelL2, X, y, LearningRate= 0.00002, c = 1, d = 10000, Device = "cuda", Verbose =0)
mse = torch.mean((Adv - X) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")

mse = torch.mean((MyGRUModel(Adv) - y) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")
UtilsTool.DisplayDetection(DetectionModelL2, [Adv])

  0%|          | 0/1000 [00:00<?, ?it/s]c:\Sources\Python311\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([77])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 1000/1000 [00:07<00:00, 136.04it/s]


RMSE: 0.01302023555475035
RMSE: 33.49793243408203
0 Detection rate 0.0



100%|██████████| 1000/1000 [00:07<00:00, 141.35it/s]


RMSE: 0.013012915220866198
RMSE: 33.505332946777344
0 Detection rate 0.0



100%|██████████| 1000/1000 [00:07<00:00, 136.53it/s]

RMSE: 0.0131248105128989
RMSE: 33.49812698364258
0 Detection rate 0.3246753215789795

